Django умеет работать с разными форматами фикстур, а по умолчанию выгружает данные в формате JSON (от англ. JavaScript Object Notation). Изначально этот формат был придуман для языка JavaScript, но оказался настолько удобным, что стал одним из стандартных форматов обмена данными; с форматом JSON умеют работать очень многие языки программирования, и Python — в их числе. 

Если выгрузить в JSON записи из таблицы `ice_cream_icecream` (за её описание отвечает модель `IceCream`), фикстуры будут выглядеть примерно так:

```json
[
{
  "model": "ice_cream.icecream",
  "pk": 1,
  "fields": {
    "is_published": true,
    "is_on_main": false,
    "title": "Классический пломбир",
    "description": "Настоящее мороженое, для истинных ценителей вкуса. Если на столе появляется пломбир — это ненадолго.",
    "wrapper": null,
    "category": 1,
    "toppings": [
      1,
      2
    ]
  }
},

{
  // Следующая запись
  "model": "ice_cream.icecream",
  "pk": 2,
  ...
}
]
```

***
## Загрузка данных из фикстуры

[Скачайте фикстуры](https://code.s3.yandex.net/backend-developer/db.json) (перейти по ссылке — нажать сочетание клавиш «Ctrl+S») и сохраните их в рабочей директории проекта «Анфиса для друзей», развёрнутого у вас на компьютере.

```
├── README.md
├── requirements.txt
├── .vscode/   
├── anfisa_for_friends/
|   ├── about/
|   ├── anfisa_for_friends/
|   ├── core/
|   ├── homepage/
|   ├── ice_cream/
|   ├── static_dev/
|   ├── templates/
|   ├── db.sqlite3      
|   ├── db.json        <-- разместите файл тут     
|   └── manage.py
└── venv/
```

Активируйте виртуальное окружение, перейдите в каталог с *manage.py* и выполните команду для загрузки фикстур в БД:

```bash
python manage.py loaddata db.json
```

> Важно: при загрузке фикстур база данных **дополняется** новыми записями. Записи, которые были в БД до загрузки фикстур, — сохраняются.

Важно: если структура БД и фикстур не совпадает — данные не будут загружены, а в консоли появится сообщение, что данные-то есть, а загрузить их некуда. Сообщение может быть, например, таким (в случае, если нет таблицы, ожидавшейся в фикстурах):

```bash
...
sqlite3.OperationalError: no such table: <имя таблицы>
...
```

***
## Создание фикстур

В Django фикстуры создаются командой `dumpdata`: после команды указывают дополнительные аргументы, а после ключа `-o` — название файла, в который нужно сохранить данные. Можно сохранить в фикстуры всю базу целиком, а можно — только выбранные модели. 

Выгружаем данные всех моделей из БД:

```bash
python manage.py dumpdata -o db.json 
``` 

Или только данные из приложения ice_cream:

```bash
python manage.py dumpdata ice_cream -o ice_cream.json 
```

А можно сохранить только данные из отдельной таблицы:

```bash
# Сохраняем данные модели icecream приложения ice_cream:
python manage.py dumpdata ice_cream.icecream -o ice_cream_icecream.json 
```
Можно экспортировать данные из нескольких моделей, для этого их названия перечисляются через пробел после команды `dumpdata`.

```bash
# Сохраняем все данные из проекта, кроме данных модели icecream приложения ice_cream:
python manage.py dumpdata --exclude ice_cream.icecream -o without_ice_cream_icecream.json
```

После выполнения этой команды в папке проекта появится файл в формате *.json*; в нём будет описание моделей и данные таблиц, связанных с этими моделями.

По умолчанию данные экспортируются в одну строку без какого-либо форматирования. За счёт этого уменьшается объём файла; программам не нужно форматирование, они и так прочтут файл. А вот человеку будет неудобно читать строку в несколько тысяч (а то и в несколько сотен тысяч) знаков. 

При экспорте данных можно указать, каким количеством отступов должны отбиваться вложенные элементы; тогда дамп будет разбит построчно, а вложенные элементы обозначены заданным количеством отступов. За такое форматирование отвечает ключ `--indent`:

```bash
python manage.py dumpdata --indent 2 -o indented_db.json 
```

***
## Засада на пути

При создании фикстур может возникнуть проблема с кодировкой; фикстуры будут созданы, но прочесть кириллицу будет невозможно:

![alt text](https://pictures.s3.yandex.net/resources/S3.2_21_1679040436.png)

С этой проблемой, скорее всего, могут столкнуться пользователи Windows.

Причина в том, что в Python при выводе данных применяет кодировку, установленную по умолчанию в операционной системе (а в Windows это кодировка `windows 1251`). Чтобы Python выводил данные в кодировке `utf-8` (именно она нужна для корректной работы) — нужно вызывать Python с ключом `-Xutf8`. 

Этот ключ относится именно к Python: он должен стоять после названия программы:

```bash
python -Xutf8 manage.py dumpdata --indent 2 -o indented_db.json 
```

Этот ключ может быть полезен во всех случаях, когда Python должен вывести какие-то данные, и среди этих данных могут быть символы кириллицы, иероглифы или эльфийские руны.

***
## Ещё одна засада

Если данные были экспортированы на одном компьютере, а залить базу надо на другом — возникнет проблема: при импорте данных будет выброшено исключение `IntegrityError`.

```bash
Exception raised when the django.db.utils.IntegrityError: Problem installing fixture <адрес фикстуры> 
: Could not load contenttypes.ContentType(pk=2): UNIQUE constraint failed: django_content_type.app_label, django_content_type.modelrelational integrity of the database is affected, e.g. a foreign key check fails, duplicate key, etc.  
```

Служебная таблица `contenttypes` создаётся и заполняется автоматически во время совершения миграции. В ней содержится информация, какие приложения и модели существуют в проекте. 

Чтобы избежать ошибки, при создании фикстуры нужно исключить таблицу `contenttypes`:

```bash
python manage.py dumpdata --exclude contenttypes -o db.json 
```

Проблемы при импорте могут возникнуть и с таблицей `auth.permission`, которая хранит информацию о правах пользователей. Эту таблицу тоже можно исключить из фикстур: `--exclude auth.permission`.

> Коротко о заполнении БД, выгрузке данных, моделях, миграциях и CRUD-операциях — в [шпаргалке](https://code.s3.yandex.net/Python-dev/cheatsheets/029-django-orm-modeli-migratsii-crud-vygruzka-v-json/029-django-orm-modeli-migratsii-crud-vygruzka-v-json.html).